# 1. Train Model

## 1.1 Build Model

In [ ]:
import os, sys
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, \
                                    Flatten, Conv3D, AveragePooling3D, MaxPooling3D, Dropout, \
                                    Concatenate, GlobalMaxPool3D, GlobalAvgPool3D
from tensorflow.keras.models import Sequential, Model, load_model
import numpy as np

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import History 

history = History()

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    verbose=1,
    patience=3,
    mode='min',
    restore_best_weights=True
)

#es = EarlyStopping(monitor='val_loss', mode='max', verbose=1, patience=10)
mc = ModelCheckpoint('/content/gdrive/My Drive/IOT_Project/best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

In [ ]:
my_model_base = keras.applications.mobilenet_v2
my_model = my_model_base.MobileNetV2


input_layer = Input(shape=(224, 224, 3), name='input_layer')
pret_model = my_model(
    input_tensor=input_layer, include_top=False, weights='imagenet', input_shape=(224, 224, 3),
    classes=3)
pret_model.trainable = False

x = pret_model.output
x = layers.GlobalAveragePooling2D()(x)
x = Dense(3, activation='softmax', name='output_fc')(x)
tmodel = Model(inputs=input_layer, outputs=x)

tmodel.compile(loss="categorical_crossentropy", optimizer="adam", metrics=[tf.keras.metrics.AUC()])

In [ ]:
tmodel.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_layer[0][0]']            
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

## 1.2. DataLoader

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 64

datagen = ImageDataGenerator(rescale = 1./255)
generator1 = datagen.flow_from_directory('/content/gdrive/My Drive/IOT_Project/data/singular/Train/Augmented', shuffle = True, target_size=(224, 224), batch_size=batch_size, class_mode = 'categorical')
generator2 = datagen.flow_from_directory('/content/gdrive/My Drive/IOT_Project/data/singular/Vali', shuffle = True, target_size=(224, 224), batch_size=batch_size, class_mode = 'categorical')

#n_img = generator.n
#steps = n_img // batch_size


Found 2594 images belonging to 3 classes.
Found 79 images belonging to 3 classes.


## 1.3. Train Model

In [ ]:
tmodel.fit(generator1, batch_size=batch_size, epochs=20, validation_data=generator2, callbacks=[early_stopping, mc, history])
tmodel.save('/content/gdrive/My Drive/cls3_64.h5')

Epoch 1/20
41/41 [==============================] - 89s 2s/step - loss: 0.6384 - auc_3: 0.8944 - val_loss: 0.5946 - val_auc_3: 0.9033
Epoch 2/20
41/41 [==============================] - 84s 2s/step - loss: 0.3215 - auc_3: 0.9795 - val_loss: 0.5582 - val_auc_3: 0.9177
Epoch 3/20
41/41 [==============================] - 83s 2s/step - loss: 0.2339 - auc_3: 0.9900 - val_loss: 0.5356 - val_auc_3: 0.9250
Epoch 4/20
41/41 [==============================] - 82s 2s/step - loss: 0.1844 - auc_3: 0.9948 - val_loss: 0.5548 - val_auc_3: 0.9240
Epoch 5/20
41/41 [==============================] - 82s 2s/step - loss: 0.1515 - auc_3: 0.9969 - val_loss: 0.5810 - val_auc_3: 0.9204
Epoch 6/20
41/41 [==============================] - 81s 2s/step - loss: 0.1270 - auc_3: 0.9979 - val_loss: 0.6124 - val_auc_3: 0.9166
Epoch 6: early stopping


# 2. Evaludate

In [ ]:
tmodel = keras.models.load_model('/content/gdrive/My Drive/cls3_64.h5', compile=False)

NameError: ignored

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

In [ ]:
generator3 = datagen.flow_from_directory('/content/gdrive/My Drive/IOT_Project/data/singular/Test/Raw/', shuffle = False, target_size=(224, 224), batch_size=batch_size, class_mode = 'categorical')
generator3.reset()
predict=tmodel.predict_generator(generator3)
y_pred = np.argmax(predict, axis=1)
y_real = generator3.labels

Found 389 images belonging to 3 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
print(roc_auc_score(y_real, predict, average='macro',multi_class='ovo'))
print(confusion_matrix(y_real, y_pred))

0.9538861162186616
[[ 43   3   1]
 [  4  14   0]
 [ 10   4 310]]


In [ ]:
steps = generator1.n // batch_size

In [ ]:
steps

40

# 3. Compile

## 3.1 Convert h5 to tflite

In [ ]:
imgs = []
for i in range(steps//2):
    a, _ = generator1.next()
    imgs.extend(a)

In [ ]:
def representative_dataset():
  for data in tf.data.Dataset.from_tensor_slices(imgs).batch(1).take(100):
    yield [tf.dtypes.cast(data, tf.float32)]

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(tmodel)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp6saiurht/assets


/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [ ]:
# Save the model.
with open('/content/gdrive/My Drive/model_cls3_64.tflite', 'wb') as f:
  f.write(tflite_quant_model)

In [ ]:
labels = '\n'.join(sorted(generator1.class_indices.keys()))

In [ ]:
labels

'Exit_arrow\nExit_here\nNon_Exit'

In [ ]:
with open('/content/gdrive/My Drive/cls3_labels.txt', 'w') as f:
  f.write(labels)

## 3.2 Compile tflite to edgetpu

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler	

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  37308      0 --:--:-- --:--:-- --:--:-- 37308
OK
deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [787 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 ht

In [ ]:
! edgetpu_compiler '/content/gdrive/My Drive/model.tflite'

/bin/bash: edgetpu_compiler: command not found
